## 05 — Export Datasets for Looker Studio
Notebook ini menyiapkan dan mengekspor dataset final yang digunakan untuk visualisasi di Looker Studio.
Dataset yang diekspor bersifat agregatif, stabil, dan siap konsumsi dashboard.
Output disimpan ke folder `data/processed`.


In [ ]:
import pandas as pd
from pathlib import Path


### Path Configuration
Notebook ini mendeteksi lokasi folder data secara relatif agar tetap konsisten saat dijalankan dari folder `notebooks` atau root project.


In [ ]:
cwd = Path.cwd()

if (cwd / "data").is_dir():
    data_path = cwd / "data" / "processed"
else:
    data_path = cwd.parent / "data" / "processed"

data_path


### Load Cleaned Base Dataset
Dataset hasil preprocessing digunakan sebagai sumber utama untuk seluruh agregasi.


In [ ]:
df = pd.read_csv(data_path / "vgsales_cleaned.csv")
df.head()


### Dataset 1. Platform Market Share
Dataset ini merepresentasikan struktur pasar berdasarkan total penjualan platform.
Struktur:
Platform | Total_Sales | Market_Share


In [ ]:
platform_market_share = (
    df.groupby("Platform", as_index=False)
      .agg(Total_Sales=("Global_Sales", "sum"))
)

platform_market_share["Market_Share"] = (
    platform_market_share["Total_Sales"]
    / platform_market_share["Total_Sales"].sum()
)

platform_market_share


### Export Platform Market Share


In [ ]:
platform_market_share.to_csv(
    data_path / "platform_market_share.csv",
    index=False
)


### Dataset 2. Genre Region Share
Dataset ini menunjukkan proporsi kontribusi genre terhadap total penjualan di masing-masing region.
Dataset disimpan dalam format long agar optimal untuk Looker Studio.
Struktur:
Genre | Region | Share


In [ ]:
genre_region = (
    df.groupby("Genre")
      .agg(
          NA=("NA_Sales", "sum"),
          EU=("EU_Sales", "sum"),
          JP=("JP_Sales", "sum"),
          Other=("Other_Sales", "sum")
      )
)

genre_region_share = genre_region.div(
    genre_region.sum(axis=0),
    axis=1
)

genre_region_share_long = (
    genre_region_share
    .reset_index()
    .melt(
        id_vars="Genre",
        var_name="Region",
        value_name="Share"
    )
)

genre_region_share_long.head()


### Export Genre Region Share


In [ ]:
genre_region_share_long.to_csv(
    data_path / "genre_region_share_long.csv",
    index=False
)


### Dataset 3. Platform Year Sales
Dataset ini digunakan untuk analisis lifecycle platform dalam bentuk time series.
Struktur:
Platform | Year | Sales


In [ ]:
platform_year_sales = (
    df.groupby(["Platform", "Year"], as_index=False)
      .agg(Sales=("Global_Sales", "sum"))
)

platform_year_sales.head()


### Export Platform Year Sales


In [ ]:
platform_year_sales.to_csv(
    data_path / "platform_year_sales_long.csv",
    index=False
)


### Dataset 4. Cleaned Base Dataset
Dataset ini diekspor ulang untuk memastikan Looker Studio memiliki satu sumber data utama yang konsisten.


In [ ]:
df.to_csv(
    data_path / "vgsales_cleaned.csv",
    index=False
)


### Export Summary
Dataset yang dihasilkan:
vgsales_cleaned.csv
platform_market_share.csv
genre_region_share_long.csv
platform_year_sales_long.csv
Keempat dataset ini mencakup kebutuhan dashboard overview, market structure, regional analysis, dan platform lifecycle.
Notebook ini menutup tahap data preparation dan agregasi untuk visualisasi.
